# 🧠 COMPASS Explainability Suite
## Three XAI Methods for Neuropsychiatric Phenotype Prediction

---

> **System**: Clinical Ontology-driven Multi-modal Predictive Agentic Support System (COMPASS)  
> **Lab**: Computational Neuroimaging Lab, IIS Biobizkaia  
> **Author**: Stijn Van Severen

---

## Why Explainability Is Non-Negotiable in Clinical AI

COMPASS predicts neuropsychiatric phenotypes (e.g., Major Depressive Disorder) from high-dimensional, multi-modal brain and clinical data. The system produces a binary classification — **CASE vs. CONTROL** — together with a probability score. But in clinical and research contexts, *a number alone is never enough*.

### The Research Case
- **Hypothesis generation**: Which biomarkers actually drive the prediction? Explainability turns a black-box score into a ranked list of candidate biomarkers for follow-up studies.
- **Cross-modal validation**: Does the model rely on neuroimaging, lipidomics, or clinical history? Explainability reveals whether the model's reasoning aligns with domain knowledge.
- **Reproducibility**: Shapley values and gradient attributions are mathematically grounded — they can be reported, compared across cohorts, and audited.

### The Clinical Case
- **Clinician trust**: A clinician cannot act on an unexplained probability. Feature attributions translate model reasoning into the language of clinical features.
- **EU MDR compliance**: Under the EU AI Act and MDR 2017/745, high-risk AI systems in healthcare must provide meaningful explanations. Explainability is a regulatory requirement, not a nice-to-have.
- **Error detection**: If the model assigns high importance to a spurious feature (e.g., a data artefact), explainability makes this visible before it causes harm.
- **Personalised medicine**: Per-patient attribution maps show *which features matter for this individual*, enabling targeted clinical follow-up.

---

## The Three XAI Methods in COMPASS

COMPASS implements three complementary, LLM-optimised explainability methods — each probing a different aspect of the prediction:

| Method | File | Role | Signal source | Paper |
|--------|------|------|--------------|-------|
| **aHFR-TokenSHAP** | `aHFR_TokenSHAP.py` | External / black-box | LLM prediction calls | Extension of [TokenSHAP (Goldshmidt et al., 2024)](https://arxiv.org/abs/2407.10114) |
| **Integrated Gradients** | `ig_attribution.py` | Internal / white-box | Local model gradients | [Sundararajan et al., 2017](https://arxiv.org/abs/1703.01365) |
| **LLM-Select** | `LLM_select.py` | Hybrid / knowledge-based | LLM domain knowledge | [Inspired by LLM-Select (Enguehard, 2024)](https://arxiv.org/abs/2407.02694) |

Together they form a **triangulation strategy**: if all three methods agree on a feature's importance, confidence is high; disagreements reveal interesting model behaviour worth investigating.

---

## 📦 Setup & Imports

Run this cell first. It adds the COMPASS root to the Python path and imports all necessary modules.

In [ ]:
import sys
import os
import json
import math
import random
import warnings
import importlib.util
from pathlib import Path
from typing import Dict, List, Set, Any, Optional

# ── Resolve paths ─────────────────────────────────────────────────────────────
# This notebook lives at: multi_agent_system/utils/validation/explainable_AI/
# COMPASS root (multi_agent_system/) is 4 levels up from the notebook file.
XAI_DIR      = Path(os.path.abspath(''))
COMPASS_ROOT = XAI_DIR.parents[3]   # .../multi_agent_system/
INFERENCE_ROOT = COMPASS_ROOT.parent  # .../INFERENCE_PIPELINE/

# Add INFERENCE_PIPELINE to sys.path so 'multi_agent_system' is importable
if str(INFERENCE_ROOT) not in sys.path:
    sys.path.insert(0, str(INFERENCE_ROOT))

print(f"COMPASS root     : {COMPASS_ROOT}")
print(f"XAI dir          : {XAI_DIR}")
print(f"sys.path[0]      : {sys.path[0]}")

# ── Core COMPASS utilities ────────────────────────────────────────────────────
from multi_agent_system.utils.core.explainability_feature_space import (
    build_feature_space,
    normalize_abs,
    aggregate_leaf_scores_to_parent,
)
from multi_agent_system.utils.core.explainability_prompt_builder import (
    build_prompt_and_spans,
    build_prompt_only,
)

# ── XAI module loader ─────────────────────────────────────────────────────────
# IMPORTANT: we must register the module in sys.modules BEFORE exec_module.
# aHFR_TokenSHAP.py uses @dataclass(frozen=True), which calls
# sys.modules[cls.__module__].__dict__ during class creation.
# Without pre-registration, this raises AttributeError on Python 3.10.

def load_xai_module(filename: str):
    module_name = filename.replace('.py', '')
    path = XAI_DIR / filename
    spec = importlib.util.spec_from_file_location(module_name, path)
    mod  = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = mod   # register BEFORE exec
    spec.loader.exec_module(mod)
    return mod

ahfr      = load_xai_module('aHFR_TokenSHAP.py')
ig_mod    = load_xai_module('ig_attribution.py')
llmselect = load_xai_module('LLM_select.py')

print("\n✅ All modules loaded successfully.")

## 🗂️ Load Pseudo-Patient Data

We use **SUBJ_001_PSEUDO** — a synthetic patient with features consistent with Major Depressive Disorder.  
The data lives in `data/pseudo_data/inputs/SUBJ_001_PSEUDO/` and contains:
- `multimodal_data.json` — hierarchical multi-modal biomarker data (lipidomics, neuroimaging, etc.)
- `non_numerical_data.txt` — free-text clinical notes

The `build_feature_space()` function converts these into a flat leaf catalogue + hierarchy tree that all three XAI methods consume.

In [ ]:
SUBJ_DIR = COMPASS_ROOT / 'data' / 'pseudo_data' / 'inputs' / 'SUBJ_001_PSEUDO'

with open(SUBJ_DIR / 'multimodal_data.json') as f:
    multimodal_data = json.load(f)

with open(SUBJ_DIR / 'non_numerical_data.txt') as f:
    non_numerical_text = f.read()

# ── Build the shared feature space ───────────────────────────────────────────
feature_space = build_feature_space(
    multimodal_data,
    non_numerical_text=non_numerical_text,
)

leaf_nodes      = feature_space['leaf_nodes']
parent_nodes    = feature_space['parent_nodes']
leaf_to_feature = feature_space['leaf_to_feature']
leaf_to_parent  = feature_space['leaf_to_parent']
hierarchy       = feature_space['hierarchy_children']
root_node       = feature_space['root_node']

print(f"Root node    : {root_node}")
print(f"Leaf features: {len(leaf_nodes)}")
print(f"Parent nodes : {len(parent_nodes)}")
print(f"Domains      : {[p for p in parent_nodes if p.startswith('dom::')]}")

---

## Method 1 — aHFR-TokenSHAP (External / Black-Box)

### What it is

**aHFR-TokenSHAP** (*Adaptive Hierarchical Frontier Randomisation — TokenSHAP*) is a Monte Carlo Shapley estimator designed specifically for LLM-based classifiers. It extends the original [TokenSHAP paper (Goldshmidt et al., 2024)](https://arxiv.org/abs/2407.10114) with three key innovations:

1. **Hierarchical frontier sampling** — instead of permuting individual features, it samples *mixed-depth frontiers* of the ontology tree (blocks can be whole domains, sub-domains, or individual leaves). This dramatically reduces the number of LLM calls needed for convergence.
2. **Adaptive calibration** — an initial calibration phase estimates primary-domain importance, then biases subsequent sampling toward informative regions of the hierarchy.
3. **Epoch-based weight updates** — sampling weights are updated between epochs using cumulative attribution evidence, making the estimator progressively more efficient.

### How it works in COMPASS

The **score function** is: for a given subset of active leaf features, build a prompt with the rest ablated to `value=__MISSING__`, call the COMPASS predictor LLM, and return `logit(P(CASE))`. The Shapley value of each leaf is its average marginal contribution to this score across many random permutations.

### Demo (offline — no LLM calls)

In this demo we use a **synthetic score function** that simulates LLM behaviour based on z-scores. This lets you run the full Shapley computation without API keys.

In [ ]:
# ── Synthetic score function (simulates LLM, no API key needed) ───────────────
# In production, this calls the COMPASS predictor LLM and returns logit(P(CASE)).
# Here we use z-scores as a proxy: features with high |z| contribute more.

rng_score = random.Random(42)

def synthetic_score_fn(active_leaves: Set[str]) -> float:
    """Simulated score: logit(P(CASE)) based on z-scores of active features."""
    if not active_leaves:
        return 0.0
    total = 0.0
    for leaf_id in active_leaves:
        feat = leaf_to_feature.get(leaf_id, {})
        z = float(feat.get('z_score', feat.get('value_numeric', 0.0)))
        total += z
    # Add small noise to simulate LLM stochasticity
    total += rng_score.gauss(0, 0.05)
    # Clamp to a reasonable logit range
    return max(-5.0, min(5.0, total / max(1, len(active_leaves)) * 2.0))

print("Score function defined.")
print(f"Score (all features active) : {synthetic_score_fn(set(leaf_nodes)):.4f}")
print(f"Score (no features active)  : {synthetic_score_fn(set()):.4f}")

In [ ]:
# ── Run aHFR-TokenSHAP ────────────────────────────────────────────────────────
# K=20 permutations, adaptive mixed-player mode, 2 independent runs for stability.

print("Running aHFR-TokenSHAP (K=20, runs=2) ...")

mean_phi, std_phi = ahfr.shapley_with_repeats(
    score_fn=synthetic_score_fn,
    feature_ids=leaf_nodes,
    hierarchy_children=hierarchy,
    root=root_node,
    leaf_ids=leaf_nodes,
    K=20,
    runs=2,
    seed=0,
    adaptive_search=True,
    mixed_players=True,
    verbose=False,
)

# Normalise to L1 for comparability
ahfr_norm   = normalize_abs(mean_phi)
ahfr_parent = normalize_abs(aggregate_leaf_scores_to_parent(mean_phi, leaf_to_parent))

print("\n✅ aHFR-TokenSHAP complete.")
print(f"   Non-zero leaf attributions: {sum(1 for v in ahfr_norm.values() if v > 0)}")

In [ ]:
# ── Display top-20 leaf features ──────────────────────────────────────────────
print("=" * 70)
print("aHFR-TokenSHAP  |  Top-20 Leaf Features by Shapley Value")
print("=" * 70)
print(f"{'Rank':<5} {'Score':>8} {'±Std':>8}  Feature")
print("-" * 70)

top_leaves = sorted(ahfr_norm.items(), key=lambda kv: kv[1], reverse=True)[:20]
for rank, (leaf_id, score) in enumerate(top_leaves, 1):
    feat = leaf_to_feature.get(leaf_id, {})
    name = feat.get('feature_name', leaf_id)[:40]
    std  = std_phi.get(leaf_id, 0.0)
    print(f"  {rank:<3} {score:>8.4f} {std:>8.4f}  {name}")

print("\n--- Domain-level aggregation ---")
print(f"{'Domain':<35} {'Score':>8}")
print("-" * 45)
for dom, score in sorted(ahfr_parent.items(), key=lambda kv: kv[1], reverse=True):
    dom_clean = dom.replace('dom::', '')
    print(f"  {dom_clean:<33} {score:>8.4f}")

---

## Method 2 — Integrated Gradients (Internal / White-Box)

### What it is

**Integrated Gradients (IG)** ([Sundararajan et al., 2017](https://arxiv.org/abs/1703.01365)) is a gradient-based attribution method with a formal axiomatic foundation. It satisfies two key axioms:

- **Sensitivity**: if a feature changes the prediction, it receives non-zero attribution.
- **Implementation invariance**: attributions depend only on the function computed, not the implementation.

The method integrates gradients along a straight-line path in embedding space from a **baseline** (a prompt with features ablated) to the **input** (the full prompt). The attribution of each token is the dot product of this integrated gradient with the embedding difference.

### How it works in COMPASS

COMPASS builds a structured prompt where each feature occupies a known character span. IG computes token-level attributions, which are then aggregated to feature-level using the span-to-token mapping. The score function is `logP(CASE) - logP(CONTROL)` from a **local** causal language model (default: `distilgpt2`).

This method is **model-internal**: it probes the local model's embedding space, not the COMPASS LLM. It is fast (no API calls), deterministic, and provides a completeness guarantee: `sum(attributions) ≈ f(input) - f(baseline)`.

> **Note**: Loading a local model requires `torch` and `transformers`. The first run downloads `distilgpt2` (~350 MB). Subsequent runs use the cache.

In [ ]:
# ── Load local model for IG ───────────────────────────────────────────────────
# Default: distilgpt2 (fast, small). Change model_name for a larger model.

IG_MODEL = 'distilgpt2'

print(f"Loading model: {IG_MODEL} ...")
model, tokenizer, device = ig_mod.load_model(model_name=IG_MODEL)
labels = ig_mod.prepare_label_tokens(tokenizer, case_str=' CASE', control_str=' CONTROL')

print(f"  Device       : {device}")
print(f"  Label tokens : CASE={labels.case_ids}  CONTROL={labels.control_ids}")
print(f"  Single-token : {labels.single_token_labels}")
print("\n✅ Model ready.")

In [ ]:
# ── Build the structured prompt with character spans ──────────────────────────
TARGET  = 'Major Depressive Disorder'
CONTROL = 'brain-implicated pathology, but NOT psychiatric'

prompt, spans = build_prompt_and_spans(
    target_condition=TARGET,
    control_condition=CONTROL,
    leaf_features=leaf_to_feature,
    active_leaf_ids=set(leaf_nodes),
)

print(f"Prompt length : {len(prompt)} characters")
print(f"Feature spans : {len(spans)} features mapped")
print("\n--- Prompt preview (first 600 chars) ---")
print(prompt[:600])
print("...")

In [ ]:
# ── Run Integrated Gradients ──────────────────────────────────────────────────
# steps=8 is fast for demonstration; use steps=50+ for publication-quality results.

print("Running Integrated Gradients (steps=8, baseline=mask) ...")
print("(This may take 30–120 s on CPU depending on prompt length)")

ig_scores, ig_debug = ig_mod.integrated_gradients_feature_importance(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    feature_spans=spans,
    labels=labels,
    device=device,
    steps=8,
    baseline_mode='mask',   # mask feature VALUE tokens → stable alignment
    span_mode='value',      # attribute only the value text, not the full line
    check_completeness=True,
    return_debug=True,
)

ig_norm   = normalize_abs(ig_scores)
ig_parent = normalize_abs(aggregate_leaf_scores_to_parent(ig_scores, leaf_to_parent))

print("\n✅ Integrated Gradients complete.")
print(f"   Prompt tokens        : {int(ig_debug['T_tokens'])}")
print(f"   Feature tokens union : {int(ig_debug['n_feature_tokens_union'])}")
if 'delta_s' in ig_debug:
    print(f"   Completeness check   : Δs={ig_debug['delta_s']:+.4f}  "
          f"Σattr={ig_debug['total_token_attr']:+.4f}  "
          f"err={ig_debug['completeness_abs_err']:.4f}")

In [ ]:
# ── Display IG results ────────────────────────────────────────────────────────
print("=" * 70)
print("Integrated Gradients  |  Top-20 Leaf Features")
print("=" * 70)
print(f"{'Rank':<5} {'Score (L1)':>12} {'Raw IG':>10}  Feature")
print("-" * 70)

top_ig = sorted(ig_norm.items(), key=lambda kv: kv[1], reverse=True)[:20]
for rank, (leaf_id, score) in enumerate(top_ig, 1):
    feat = leaf_to_feature.get(leaf_id, {})
    name = feat.get('feature_name', leaf_id)[:40]
    raw  = ig_scores.get(leaf_id, 0.0)
    print(f"  {rank:<3} {score:>12.6f} {raw:>+10.4f}  {name}")

print("\n--- Domain-level aggregation ---")
for dom, score in sorted(ig_parent.items(), key=lambda kv: kv[1], reverse=True):
    dom_clean = dom.replace('dom::', '')
    print(f"  {dom_clean:<35} {score:.4f}")

---

## Method 3 — LLM-Select (Hybrid / Knowledge-Based)

### What it is

**LLM-Select** ([Enguehard, 2024](https://arxiv.org/abs/2407.02694)) uses an LLM's **prior domain knowledge** — rather than patient-specific data — to score feature importance. The LLM is asked: *"Given the task of predicting [phenotype], how important is each feature?"* and returns integer scores (1–1000) for every leaf and parent feature.

### Why this is valuable

- **Zero data required**: LLM-Select does not need the patient's actual values. It reflects what the model *knows* about the phenotype from pre-training.
- **Sanity check**: If LLM-Select and aHFR-TokenSHAP agree on top features, the model's reasoning is consistent with biomedical literature.
- **Prior elicitation**: In small-cohort research, LLM-Select provides a literature-informed prior that can guide feature selection before running expensive experiments.
- **Complementarity**: LLM-Select captures *general* feature relevance; aHFR-TokenSHAP captures *patient-specific* relevance. Both are needed.

### How it works in COMPASS

The method calls an OpenAI-compatible API (OpenRouter or direct OpenAI) with a structured JSON output schema. Multiple repeats are averaged to reduce sampling variance.

> **Requires**: `OPENAI_API_KEY` or `OPENROUTER_API_KEY` in your `.env` file.  
> The demo below shows the full workflow. If no key is available, a mock result is shown.

In [ ]:
# ── LLM-Select: check for API key ─────────────────────────────────────────────
from dotenv import load_dotenv
load_dotenv(COMPASS_ROOT / '.env')

OPENAI_KEY     = os.environ.get('OPENAI_API_KEY', '')
OPENROUTER_KEY = os.environ.get('OPENROUTER_API_KEY', '')

HAS_API_KEY = bool(OPENAI_KEY or OPENROUTER_KEY)
print(f"OpenAI key available    : {bool(OPENAI_KEY)}")
print(f"OpenRouter key available: {bool(OPENROUTER_KEY)}")

if not HAS_API_KEY:
    print("\n⚠️  No API key found. A mock result will be shown instead.")
    print("   Add OPENAI_API_KEY or OPENROUTER_API_KEY to your .env file to run live.")

In [ ]:
# ── Run LLM-Select (or mock) ──────────────────────────────────────────────────
# Use a small subset of features for the demo to keep the prompt manageable.
# In production, all leaf_nodes and parent_nodes are passed.

DEMO_LEAF_IDS   = leaf_nodes[:30]   # first 30 leaves for demo
DEMO_PARENT_IDS = parent_nodes[:10] # first 10 parents

if HAS_API_KEY:
    client_kwargs = {}
    if OPENROUTER_KEY:
        client_kwargs = {
            'api_key': OPENROUTER_KEY,
            'base_url': 'https://openrouter.ai/api/v1',
        }

    print("Running LLM-Select (repeats=2, model=gpt-4o-mini) ...")
    try:
        leaf_runs, parent_runs = llmselect.get_llm_select_scores(
            phenotype=TARGET,
            leaf_ids=DEMO_LEAF_IDS,
            parent_ids=DEMO_PARENT_IDS,
            repeats=2,
            model='gpt-4o-mini',
            temperature=0.7,
            client_kwargs=client_kwargs,
        )
        llmsel_leaf = {
            lid: sum(r['norm'].get(lid, 0.0) for r in leaf_runs) / len(leaf_runs)
            for lid in DEMO_LEAF_IDS
        }
        llmsel_parent = {
            pid: sum(r['norm'].get(pid, 0.0) for r in parent_runs) / len(parent_runs)
            for pid in DEMO_PARENT_IDS
        }
        print("\n✅ LLM-Select complete (live API).")
        LLMSEL_LIVE = True
    except Exception as e:
        print(f"\n⚠️  LLM-Select API call failed: {e}")
        LLMSEL_LIVE = False
else:
    LLMSEL_LIVE = False

if not LLMSEL_LIVE:
    # ── Mock: simulate LLM scores using z-score magnitude as proxy ────────────
    print("Generating mock LLM-Select scores (z-score proxy) ...")
    rng_mock = random.Random(99)
    llmsel_leaf = {}
    for lid in DEMO_LEAF_IDS:
        feat = leaf_to_feature.get(lid, {})
        z = abs(float(feat.get('z_score', feat.get('value_numeric', 0.0))))
        llmsel_leaf[lid] = z + rng_mock.uniform(0, 0.5)
    total = sum(llmsel_leaf.values()) or 1.0
    llmsel_leaf = {k: v / total for k, v in llmsel_leaf.items()}

    llmsel_parent = {}
    for pid in DEMO_PARENT_IDS:
        llmsel_parent[pid] = rng_mock.uniform(0.05, 0.3)
    total_p = sum(llmsel_parent.values()) or 1.0
    llmsel_parent = {k: v / total_p for k, v in llmsel_parent.items()}

    print("✅ Mock LLM-Select scores generated.")

In [ ]:
# ── Display LLM-Select results ────────────────────────────────────────────────
print("=" * 70)
print(f"LLM-Select  |  Top-20 Leaf Features  {'(LIVE)' if LLMSEL_LIVE else '(MOCK)'}")
print("=" * 70)
print(f"{'Rank':<5} {'Score (L1)':>12}  Feature")
print("-" * 70)

top_llmsel = sorted(llmsel_leaf.items(), key=lambda kv: kv[1], reverse=True)[:20]
for rank, (leaf_id, score) in enumerate(top_llmsel, 1):
    feat = leaf_to_feature.get(leaf_id, {})
    name = feat.get('feature_name', leaf_id)[:45]
    print(f"  {rank:<3} {score:>12.6f}  {name}")

print("\n--- Domain-level scores ---")
for dom, score in sorted(llmsel_parent.items(), key=lambda kv: kv[1], reverse=True):
    dom_clean = dom.replace('dom::', '')
    print(f"  {dom_clean:<35} {score:.4f}")

---

## 🔍 Cross-Method Comparison

The real power of the COMPASS XAI suite lies in **triangulation**: comparing all three methods reveals where they agree (high-confidence features) and where they diverge (interesting model behaviour).

- **Agreement** → feature is robustly important across signal sources
- **aHFR high, IG low** → feature matters for LLM reasoning but not for local model gradients (may reflect LLM-specific knowledge)
- **LLM-Select high, aHFR low** → feature is known to be relevant in literature but does not drive *this patient's* prediction
- **IG high, others low** → feature has strong local embedding signal but may be a prompt artefact

In [ ]:
# ── Build comparison table over shared leaf IDs ───────────────────────────────
shared_ids = [lid for lid in DEMO_LEAF_IDS if lid in ahfr_norm and lid in ig_norm]

rows = []
for lid in shared_ids:
    feat = leaf_to_feature.get(lid, {})
    name = feat.get('feature_name', lid)[:35]
    a = ahfr_norm.get(lid, 0.0)
    b = ig_norm.get(lid, 0.0)
    c = llmsel_leaf.get(lid, 0.0)
    avg = (a + b + c) / 3.0
    rows.append((avg, name, a, b, c))

rows.sort(reverse=True)

print("=" * 85)
print("Cross-Method Comparison  |  Shared Features  (sorted by mean score)")
print("=" * 85)
print(f"{'Feature':<37} {'aHFR':>8} {'IG':>8} {'LLMSel':>8} {'Mean':>8}")
print("-" * 85)
for avg, name, a, b, c in rows[:25]:
    print(f"  {name:<35} {a:>8.4f} {b:>8.4f} {c:>8.4f} {avg:>8.4f}")

print("\nLegend: aHFR=aHFR-TokenSHAP  IG=Integrated Gradients  LLMSel=LLM-Select")

In [ ]:
# ── Rank correlation between methods ─────────────────────────────────────────
# Spearman rank correlation: 1.0 = perfect agreement, 0.0 = no agreement.

def spearman_rho(scores_a: dict, scores_b: dict) -> float:
    keys = sorted(set(scores_a) & set(scores_b))
    if len(keys) < 2:
        return float('nan')
    def ranks(d):
        sorted_keys = sorted(keys, key=lambda k: d.get(k, 0.0), reverse=True)
        return {k: i+1 for i, k in enumerate(sorted_keys)}
    ra, rb = ranks(scores_a), ranks(scores_b)
    n = len(keys)
    d2 = sum((ra[k] - rb[k])**2 for k in keys)
    return 1.0 - 6.0 * d2 / (n * (n**2 - 1))

rho_ahfr_ig     = spearman_rho(ahfr_norm,  ig_norm)
rho_ahfr_llmsel = spearman_rho(ahfr_norm,  llmsel_leaf)
rho_ig_llmsel   = spearman_rho(ig_norm,    llmsel_leaf)

print("Spearman Rank Correlation between XAI Methods")
print("=" * 45)
print(f"  aHFR-TokenSHAP  ↔  IG              : {rho_ahfr_ig:+.3f}")
print(f"  aHFR-TokenSHAP  ↔  LLM-Select      : {rho_ahfr_llmsel:+.3f}")
print(f"  IG              ↔  LLM-Select       : {rho_ig_llmsel:+.3f}")
print()
print("Interpretation:")
print("  > 0.6  → strong agreement (feature ranking is consistent)")
print("  0.3–0.6 → moderate agreement")
print("  < 0.3  → methods probe different aspects of the model")

---

## 🔗 Integration with the COMPASS Pipeline

In production, all three methods are orchestrated automatically by `utils/core/explainability_runner.py`. The runner is invoked from `main.py` via the `--xai_methods` flag:

```bash
python main.py data/pseudo_data/inputs/SUBJ_001_PSEUDO \
  --target "Major Depressive Disorder" \
  --control "brain-implicated pathology, but NOT psychiatric" \
  --backend openrouter \
  --xai_methods external,internal,hybrid
```

The runner:
1. Calls `build_feature_space()` to create the shared leaf catalogue and hierarchy.
2. Selects the best COMPASS prediction attempt (highest critic score).
3. Runs each requested method in sequence.
4. Saves results to `xai_feature_importance_{participant_id}.json`.
5. Passes results to the **Communicator** agent, which generates a human-readable `xai_explainability_report.md`.

The cell below demonstrates the data flow using the pseudo-patient data.

In [ ]:
# ── COMPASS runner data flow summary ─────────────────────────────────────────
print("COMPASS XAI Runner — Data Flow Summary")
print("=" * 60)
print()
print("Step 1: build_feature_space()")
print(f"  Input  : multimodal_data.json + non_numerical_data.txt")
print(f"  Output : {len(leaf_nodes)} leaf features, {len(parent_nodes)} parent nodes")
print()
print("Step 2: run_explainability_methods()")
print("  external → aHFR_TokenSHAP.py  (monte_carlo_hfr_tokenshap / shapley_with_repeats)")
print("  internal → ig_attribution.py  (integrated_gradients_feature_importance)")
print("  hybrid   → LLM_select.py      (get_llm_select_scores)")
print()
print("Step 3: Output artifacts")
print("  xai_feature_importance_{participant_id}.json")
print("  xai_explainability_report.md  (generated by Communicator agent)")
print()
print("Feature space summary for SUBJ_001_PSEUDO:")
domain_counts = {}
for lid in leaf_nodes:
    dom = leaf_to_feature.get(lid, {}).get('domain', 'UNKNOWN')
    domain_counts[dom] = domain_counts.get(dom, 0) + 1
for dom, cnt in sorted(domain_counts.items(), key=lambda x: -x[1]):
    print(f"  {dom:<35} {cnt:>4} features")

---

## 📋 Summary & Method Comparison

| Property | aHFR-TokenSHAP | Integrated Gradients | LLM-Select |
|----------|---------------|---------------------|------------|
| **Signal source** | LLM prediction calls | Local model gradients | LLM prior knowledge |
| **Requires API** | Yes (predictor LLM) | No (local model) | Yes (scoring LLM) |
| **Patient-specific** | ✅ Yes | ✅ Yes | ❌ No (phenotype-level) |
| **Theoretical basis** | Shapley values (game theory) | Axiomatic attribution | LLM knowledge elicitation |
| **Hierarchy-aware** | ✅ Yes (HFR) | ❌ No (token-level) | ✅ Yes (parent + leaf) |
| **Computational cost** | High (many LLM calls) | Medium (local fwd/bwd) | Low (few LLM calls) |
| **Key parameter** | K (permutations) | steps (integration steps) | repeats |
| **Output** | Leaf Shapley values ± std | Token/feature attributions | Integer importance scores |
| **Completeness axiom** | ✅ (in expectation) | ✅ (exact) | ❌ (heuristic) |

### Recommended usage

- **Research**: Run all three. Report Spearman correlation as a consistency metric. Highlight features where all three agree.
- **Clinical report**: Use aHFR-TokenSHAP (patient-specific, LLM-grounded) as the primary attribution. Use LLM-Select as a literature-informed sanity check.
- **Debugging**: Use IG to inspect whether the local model's embedding space encodes clinically meaningful signals.

---

## 📚 References

1. **TokenSHAP**: Goldshmidt, R., et al. (2024). *TokenSHAP: Interpreting Large Language Models with Monte Carlo Shapley Value Estimation*. arXiv:2407.10114. https://arxiv.org/abs/2407.10114

2. **Integrated Gradients**: Sundararajan, M., Taly, A., & Yan, Q. (2017). *Axiomatic Attribution for Deep Networks*. ICML 2017. arXiv:1703.01365. https://arxiv.org/abs/1703.01365

3. **LLM-Select**: Enguehard, J. (2024). *LLM-Select: Feature Selection with Large Language Models*. arXiv:2407.02694. https://arxiv.org/abs/2407.02694

4. **COMPASS-engine**: Van Severen, S., Diez, I. P., & Cortes, J. M. (2026). *COMPASS: Clinical ontology-driven multi-modal predictive agentic support system*. Manuscript in preparation.